## Imports

In [ ]:
!pip install transformers
!pip install datasets

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
import json

import tensorflow as tf
from tensorflow import keras
from transformers import AutoTokenizer, DataCollatorWithPadding, TFAutoModelForSequenceClassification
from datasets import Dataset

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from scipy.spatial.distance import cosine

# For model evaluation:
def cosine_score(predicted_values, true_values):
    return 1 - cosine(predicted_values, true_values)

# EDA

## Get Data

In [ ]:
# Get data from here:
!git clone https://bitbucket.org/ssix-project/semeval-2017-task-5-subtask-2.git

## Examine data

In [ ]:
def __text_company(all_data):
    
    text = []
    company = []
    ids = []
    for data in all_data:
        text.append(data['title'])
        company.append(data['company'])
        ids.append(data['id'])
    return text, company, ids


def __text_sentiment_company(all_data):

    text = []
    sentiment = []
    company = []
    for data in all_data:
        text.append(data['title'])
        company.append(data['company'])
        # This field does not exist in test dataset
        if 'sentiment' in data:
            sentiment.append(data['sentiment'])
        elif 'sentiment score' in data:
            sentiment.append(data['sentiment score'])
    return text, np.asarray(sentiment), company

def fin_data(data_type, test_data=False):

    with open("semeval-2017-task-5-subtask-2/" + data_type, 'r') as fp:
        if test_data:
            return __text_company(json.load(fp))
        return __text_sentiment_company(json.load(fp))

In [ ]:
train_texts, train_sentiments, train_companies = fin_data('Headline_Trainingdata.json')
test_texts, test_sentiments, test_companies = fin_data('Headlines_Testdata_withscores.json')

df = pd.DataFrame(list(zip(train_texts, train_sentiments, train_companies)), columns =['text', 'Sentiment', 'company'])
df = df.reset_index()
df.rename({'index':'input_ids'}, axis = 1, inplace=True)
df.head()

,input_ids,text,Sentiment,company
0,0,Morrisons book second consecutive quarter of sales growth,0.430,Morrisons
1,1,IMI posts drop in first-quarter organic revenue; warns on full year,-0.344,IMI
2,2,"Glencore to refinance its short-term debt early, shares rise",0.340,Glencore
3,3,EasyJet attracts more passengers in June but still lags Ryanair,0.259,Ryanair
4,4,Barclays 'bad bank' chief to step down,-0.231,Barclays


## Preprocessing data for input into BERT

In [ ]:
raw_train_ds = Dataset.from_json("semeval-2017-task-5-subtask-2/Headline_Trainingdata.json")
raw_test_ds = Dataset.from_json("semeval-2017-task-5-subtask-2/Headlines_Testdata_withscores.json")
raw_datasets = {"train": raw_train_ds, "test": raw_test_ds}

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-35be68298e7ca5ec/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-891fd242f4daf689/0.0.0. Subsequent calls will reuse this data.


In [ ]:
raw_train_ds

Dataset({
    features: ['id', 'company', 'title', 'sentiment'],
    num_rows: 1142
})

In [ ]:
raw_train_ds[0]

{'id': 2,
 'company': 'Morrisons',
 'title': 'Morrisons book second consecutive quarter of sales growth',
 'sentiment': 0.43}

## Tokenize datasets

#### Get Max Tokenized Vector Length

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def get_max_tokenized_length(ds):
    max_len_raw = 0
    max_len_token_vector = 0
    
    for text_input in ds:
        
        if len(text_input.split(" ")) > max_len_raw:
            max_len_raw = len(text_input.split(" "))
        
        if len(tokenizer(text_input)['input_ids']) > max_len_token_vector:
            max_len_token_vector = len(tokenizer(text_input)['input_ids'])

    return max_len_raw, max_len_token_vector

max_len_raw_train, max_len_token_vector_train = get_max_tokenized_length(raw_train_ds['title'])
max_len_raw_test, max_len_token_vector_test = get_max_tokenized_length(raw_test_ds['title'])

print(f'max_len_raw_train = {max_len_raw_train}')
print(f'max_len_token_vector_train = {max_len_token_vector_train}')
print('')
print(f'max_len_raw_test = {max_len_raw_test}')
print(f'max_len_token_vector_test = {max_len_token_vector_test}')
print('')
max_len_token_vector = max(max_len_token_vector_train, max_len_token_vector_test)
print(f'max_len_token_vector (BOTH) = {max_len_token_vector}')

max_len_raw_train = 18
max_len_token_vector_train = 29

max_len_raw_test = 12
max_len_token_vector_test = 21

max_len_token_vector (BOTH) = 29


#### Tokenize and save as TF Dataset object

In [ ]:
def preprocess_function(examples):
    global max_len_token_vector
    label = examples["sentiment"] 
    examples = tokenizer(examples["title"], truncation=True, padding="max_length", max_length=max_len_token_vector)
    
    # Change this to real number
    examples["label"] = float(label)
    return examples


for split in raw_datasets:
    raw_datasets[split] = raw_datasets[split].map(preprocess_function, remove_columns=['id', 'company', 'title', 'sentiment'])

tokenized_datasets = raw_datasets.copy()

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

  0%|          | 0/1142 [00:00<?, ?ex/s]

  0%|          | 0/14 [00:00<?, ?ex/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


#### Look at tokenized dataset

In [ ]:
# Full training set
np.asarray(raw_datasets['train']['input_ids'])

array([[  101,  9959,  2015, ...,     0,     0,     0],
       [  101, 10047,  2072, ...,     0,     0,     0],
       [  101,  8904, 17345, ...,     0,     0,     0],
       ...,
       [  101,  2332,  7529, ...,     0,     0,     0],
       [  101,  3316,  6494, ...,     0,     0,     0],
       [  101,  3115, 12443, ...,     0,     0,     0]])

In [ ]:
# First sentence observation -- tokenized representation
np.asarray(raw_datasets['train']['input_ids'])[0,:]

array([ 101, 9959, 2015, 2338, 2117, 5486, 4284, 1997, 4341, 3930,  102,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [ ]:
# (training_set_rows, max_token_vector_length)
np.asarray(raw_datasets['train']['input_ids']).shape

(1142, 29)

In [ ]:
# Convert back from tokenized to text representation
tokenizer.decode(tokenizer(raw_train_ds['title'][6])['input_ids'])

'[CLS] bilfinger industrial services win a£100m bp contract extension [SEP]'

In [ ]:
# Same, represented as tokens
# 101 - [CLS]
# 102 - [SEP]
tokenizer(raw_train_ds['title'][6])['input_ids']

[101,
 12170,
 10270,
 9912,
 3919,
 2578,
 2663,
 1037,
 29646,
 18613,
 2213,
 17531,
 3206,
 5331,
 102]

In [ ]:
# Batch #1:


# [tokenized sentences,
#  N/A (not used by BERT),
#  attention masks,
#  labels]

arr = tf_train_dataset.as_numpy_iterator()
l = list(arr)
l[0]

({'input_ids': array([[  101,  2414,  2851, 27918,  1024, 26236,  9818,  1998,  3115,
          12443,  6661,  4125,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  101, 16565, 12058,  2004,  4636,  3208,  8040,  8093, 27381,
           2015,  4311,  1037,  2501,  2095,   102,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  101,  6031, 28305,  2102,  1005,  1055, 15570,  6045, 14238,
           4710, 12174,  5618, 14523,  2471,  2028,  2353,   102,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  101, 10651,  1015,  1011, 11825, 11146, 12386,  1037,  2353,
           1997,  9959,  2015,  6501,  3206,   102,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [ 

# BERT
bert-base-uncased  
Cross Validation cosine_similarity = 74.94

## Preprocessing for input into BERT

In [ ]:
# Same as above
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def get_max_tokenized_length(ds):
    max_len_raw = 0
    max_len_token_vector = 0
    
    for text_input in ds:
        
        if len(text_input.split(" ")) > max_len_raw:
            max_len_raw = len(text_input.split(" "))
        
        if len(tokenizer(text_input)['input_ids']) > max_len_token_vector:
            max_len_token_vector = len(tokenizer(text_input)['input_ids'])

    return max_len_raw, max_len_token_vector

max_len_raw_train, max_len_token_vector_train = get_max_tokenized_length(raw_train_ds['title'])
max_len_raw_test, max_len_token_vector_test = get_max_tokenized_length(raw_test_ds['title'])

max_len_token_vector = max(max_len_token_vector_train, max_len_token_vector_test)

In [ ]:
# Same as above
def preprocess_function(examples):
    global max_len_token_vector
    label = examples["sentiment"] 
    examples = tokenizer(examples["title"], truncation=True, padding="max_length", max_length=max_len_token_vector)
    
    # Change this to real number
    examples["label"] = float(label)
    return examples

## Perform 10-Fold Cross Validation

In [ ]:
def tenfold_cross_val(train_ds):

    all_results = []

    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    for i, (train_index, test_index) in enumerate(kfold.split(train_ds['title'], train_ds['sentiment'])):
        
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")

        tokenized_cv_train = Dataset.from_dict(train_ds[train_index]).map(preprocess_function, remove_columns=['id', 'company', 'title', 'sentiment'])
        tokenized_cv_test = Dataset.from_dict(train_ds[test_index]).map(preprocess_function, remove_columns=['id', 'company', 'title', 'sentiment'])

        data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

        cv_train_dataset_tf = tokenized_cv_train.to_tf_dataset(
            columns=["attention_mask", "input_ids", "token_type_ids"],
            label_cols=["labels"],
            shuffle=False,
            collate_fn=data_collator,
            batch_size=4,
        )

        cv_test_dataset_tf = tokenized_cv_test.to_tf_dataset(
            columns=["attention_mask", "input_ids", "token_type_ids"],
            label_cols=["labels"],
            shuffle=False,
            collate_fn=data_collator,
            batch_size=4,
        )
    
        model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1) # num_labels=1 --> regression head after BERT layer (linear layer for output)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss="mse", metrics=['cosine_proximity'])
        model.fit(cv_train_dataset_tf, epochs=10)

        predicted_sentiments = model.predict(cv_test_dataset_tf)
        result = cosine_score(predicted_sentiments['logits'][:,0],  np.concatenate([y for x, y in cv_test_dataset_tf], axis=0))
        print(f'Fold {i} cosine_similarity = {result}\n\n')
        all_results.append(result)

    return all_results

In [ ]:
cv_raw_data_cosine_similarity_all_folds = tenfold_cross_val(raw_train_ds)
cv_raw_data_cosine_similarity_avg = (sum(cv_raw_data_cosine_similarity_all_folds) / len(cv_raw_data_cosine_similarity_all_folds)) * 100

Fold 0:
  Train: index=[   0    1    2 ... 1139 1140 1141]
  Test:  index=[  10   23   31   44   51   56   70   86   88   96  101  109  113  128
  140  158  178  198  199  208  240  243  244  247  277  291  292  296
  298  306  318  319  323  331  333  336  342  344  355  361  362  367
  371  377  388  390  398  422  429  439  442  447  448  451  453  461
  482  493  494  497  514  528  529  530  535  538  558  599  605  622
  644  666  674  701  703  708  711  718  727  733  741  754  757  771
  774  777  793  810  814  827  844  875  893  909  922  924  933  940
  961  970  976 1001 1008 1011 1014 1046 1062 1077 1080 1094 1096 1107
 1119 1128 1133]


  0%|          | 0/1027 [00:00<?, ?ex/s]

  0%|          | 0/115 [00:00<?, ?ex/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
257/257 [==============================] - 42s 103ms/step - loss: 0.1343 - cosine_proximity: 0.3914
Epoch 2/10
257/257 [==============================] - 26s 100ms/step - loss: 0.0619 - cosine_proximity: 0.6816
Epoch 3/10
257/257 [==============================] - 25s 98ms/step - loss: 0.0366 - cosine_proximity: 0.7537
Epoch 4/10
257/257 [==============================] - 21s 80ms/step - loss: 0.0298 - cosine_proximity: 0.7712
Epoch 5/10
257/257 [==============================] - 23s 91ms/step - loss: 0.0225 - cosine_proximity: 0.8179
Epoch 6/10
257/257 [==============================] - 16s 62ms/step - loss: 0.0196 - cosine_proximity: 0.8491
Epoch 7/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0199 - cosine_proximity: 0.8374
Epoch 8/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0206 - cosine_proximity: 0.8530
Epoch 9/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0191 - cosine_proximity: 0.8452
Epoch 10

...

Epoch 1/10
257/257 [==============================] - 29s 53ms/step - loss: 0.1225 - cosine_proximity: 0.4076
Epoch 2/10
257/257 [==============================] - 14s 53ms/step - loss: 0.0593 - cosine_proximity: 0.7033
Epoch 3/10
257/257 [==============================] - 14s 53ms/step - loss: 0.0324 - cosine_proximity: 0.8025
Epoch 4/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0257 - cosine_proximity: 0.8025
Epoch 5/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0227 - cosine_proximity: 0.8337
Epoch 6/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0234 - cosine_proximity: 0.8239
Epoch 7/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0202 - cosine_proximity: 0.8278
Epoch 8/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0164 - cosine_proximity: 0.8395
Epoch 9/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0150 - cosine_proximity: 0.8531
Epoch 10/1

  0%|          | 0/1028 [00:00<?, ?ex/s]

  0%|          | 0/114 [00:00<?, ?ex/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
257/257 [==============================] - 28s 50ms/step - loss: 0.1098 - cosine_proximity: 0.4650
Epoch 2/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0578 - cosine_proximity: 0.7237
Epoch 3/10
257/257 [==============================] - 13s 50ms/step - loss: 0.0413 - cosine_proximity: 0.7588
Epoch 4/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0289 - cosine_proximity: 0.8249
Epoch 5/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0234 - cosine_proximity: 0.8502
Epoch 6/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0191 - cosine_proximity: 0.8444
Epoch 7/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0175 - cosine_proximity: 0.8405
Epoch 8/10
257/257 [==============================] - 13s 50ms/step - loss: 0.0184 - cosine_proximity: 0.8366
Epoch 9/10
257/257 [==============================] - 13s 50ms/step - loss: 0.0178 - cosine_proximity: 0.8444
Epoch 10/1

In [ ]:
cv_raw_data_cosine_similarity_avg

74.94607746601105

# finBERT
ProsusAI/finbert  
Cross Validation cosine_similarity = 77.79

## Preprocessing for input into finBERT

In [5]:
checkpoint = "ProsusAI/finbert"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def get_max_tokenized_length(ds):
    max_len_raw = 0
    max_len_token_vector = 0
    
    for text_input in ds:
        
        if len(text_input.split(" ")) > max_len_raw:
            max_len_raw = len(text_input.split(" "))
        
        if len(tokenizer(text_input)['input_ids']) > max_len_token_vector:
            max_len_token_vector = len(tokenizer(text_input)['input_ids'])

    return max_len_raw, max_len_token_vector

max_len_raw_train, max_len_token_vector_train = get_max_tokenized_length(raw_train_ds['title'])
max_len_raw_test, max_len_token_vector_test = get_max_tokenized_length(raw_test_ds['title'])

print(f'max_len_raw_train = {max_len_raw_train}')
print(f'max_len_token_vector_train = {max_len_token_vector_train}')
print('')
print(f'max_len_raw_test = {max_len_raw_test}')
print(f'max_len_token_vector_test = {max_len_token_vector_test}')
print('')
max_len_token_vector = max(max_len_token_vector_train, max_len_token_vector_test)
print(f'max_len_token_vector (BOTH) = {max_len_token_vector}')

max_len_raw_train = 18
max_len_token_vector_train = 29

max_len_raw_test = 12
max_len_token_vector_test = 21

max_len_token_vector (BOTH) = 29


In [12]:
tokenizer

BertTokenizerFast(name_or_path='ProsusAI/finbert', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [7]:
def preprocess_function(examples):
    global max_len_token_vector
    label = examples["sentiment"] 
    examples = tokenizer(examples["title"], truncation=True, padding="max_length", max_length=max_len_token_vector)
    
    # Change this to real number
    examples["label"] = float(label)
    return examples

## Perform 10-Fold Cross Validation

In [13]:
def tenfold_cross_val(train_ds, checkpoint):

    all_results = []

    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    for i, (train_index, test_index) in enumerate(kfold.split(train_ds['title'], train_ds['sentiment'])):
        
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")

        tokenized_cv_train = Dataset.from_dict(train_ds[train_index]).map(preprocess_function, remove_columns=['id', 'company', 'title', 'sentiment'])
        tokenized_cv_test = Dataset.from_dict(train_ds[test_index]).map(preprocess_function, remove_columns=['id', 'company', 'title', 'sentiment'])

        data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

        cv_train_dataset_tf = tokenized_cv_train.to_tf_dataset(
            columns=["attention_mask", "input_ids", "token_type_ids"],
            label_cols=["labels"],
            shuffle=False,
            collate_fn=data_collator,
            batch_size=4,
        )

        cv_test_dataset_tf = tokenized_cv_test.to_tf_dataset(
            columns=["attention_mask", "input_ids", "token_type_ids"],
            label_cols=["labels"],
            shuffle=False,
            collate_fn=data_collator,
            batch_size=4,
        )
    
        model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1) # num_labels=1 --> regression head after BERT layer (linear layer for output)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss="mse", metrics=['cosine_proximity'])
        model.fit(cv_train_dataset_tf, epochs=10)

        predicted_sentiments = model.predict(cv_test_dataset_tf)
        result = cosine_score(predicted_sentiments['logits'][:,0],  np.concatenate([y for x, y in cv_test_dataset_tf], axis=0))
        print(f'Fold {i} cosine_similarity = {result}\n\n')
        all_results.append(result)

    return all_results

In [14]:
cv_raw_data_cosine_similarity_all_folds = tenfold_cross_val(raw_train_ds, checkpoint)
cv_raw_data_cosine_similarity_avg = (sum(cv_raw_data_cosine_similarity_all_folds) / len(cv_raw_data_cosine_similarity_all_folds)) * 100

Fold 0:
  Train: index=[   0    1    2 ... 1139 1140 1141]
  Test:  index=[  10   23   31   44   51   56   70   86   88   96  101  109  113  128
  140  158  178  198  199  208  240  243  244  247  277  291  292  296
  298  306  318  319  323  331  333  336  342  344  355  361  362  367
  371  377  388  390  398  422  429  439  442  447  448  451  453  461
  482  493  494  497  514  528  529  530  535  538  558  599  605  622
  644  666  674  701  703  708  711  718  727  733  741  754  757  771
  774  777  793  810  814  827  844  875  893  909  922  924  933  940
  961  970  976 1001 1008 1011 1014 1046 1062 1077 1080 1094 1096 1107
 1119 1128 1133]


  0%|          | 0/1027 [00:00<?, ?ex/s]

  0%|          | 0/115 [00:00<?, ?ex/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
257/257 [==============================] - 32s 52ms/step - loss: 0.1015 - cosine_proximity: 0.5278
Epoch 2/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0515 - cosine_proximity: 0.7186
Epoch 3/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0348 - cosine_proximity: 0.7712
Epoch 4/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0263 - cosine_proximity: 0.8082
Epoch 5/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0240 - cosine_proximity: 0.8374
Epoch 6/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0223 - cosine_proximity: 0.7868
Epoch 7/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0226 - cosine_proximity: 0.8140
Epoch 8/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0184 - cosine_proximity: 0.8296
Epoch 9/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0161 - cosine_proximity: 0.8491
Epoch 10/1

...

Epoch 1/10
257/257 [==============================] - 28s 52ms/step - loss: 0.1123 - cosine_proximity: 0.4582
Epoch 2/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0605 - cosine_proximity: 0.6936
Epoch 3/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0423 - cosine_proximity: 0.7305
Epoch 4/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0294 - cosine_proximity: 0.7909
Epoch 5/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0262 - cosine_proximity: 0.7986
Epoch 6/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0251 - cosine_proximity: 0.8161
Epoch 7/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0225 - cosine_proximity: 0.8161
Epoch 8/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0211 - cosine_proximity: 0.8161
Epoch 9/10
257/257 [==============================] - 14s 53ms/step - loss: 0.0198 - cosine_proximity: 0.8414
Epoch 10/1

  0%|          | 0/1028 [00:00<?, ?ex/s]

  0%|          | 0/114 [00:00<?, ?ex/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
257/257 [==============================] - 29s 52ms/step - loss: 0.1291 - cosine_proximity: 0.3619
Epoch 2/10
257/257 [==============================] - 14s 53ms/step - loss: 0.0593 - cosine_proximity: 0.7062
Epoch 3/10
257/257 [==============================] - 14s 53ms/step - loss: 0.0471 - cosine_proximity: 0.7276
Epoch 4/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0317 - cosine_proximity: 0.8054
Epoch 5/10
257/257 [==============================] - 14s 53ms/step - loss: 0.0271 - cosine_proximity: 0.8113
Epoch 6/10
257/257 [==============================] - 14s 53ms/step - loss: 0.0205 - cosine_proximity: 0.8521
Epoch 7/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0197 - cosine_proximity: 0.8385
Epoch 8/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0195 - cosine_proximity: 0.8560
Epoch 9/10
257/257 [==============================] - 14s 53ms/step - loss: 0.0182 - cosine_proximity: 0.8638
Epoch 10/1

In [15]:
cv_raw_data_cosine_similarity_avg

77.78641879558563

# roBERTa
roberta-base  
Cross Validation cosine_similarity = 77.61

## Preprocessing for input into roBERTa

In [16]:
checkpoint = "roberta-base"

In [17]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def get_max_tokenized_length(ds):
    max_len_raw = 0
    max_len_token_vector = 0
    
    for text_input in ds:
        
        if len(text_input.split(" ")) > max_len_raw:
            max_len_raw = len(text_input.split(" "))
        
        if len(tokenizer(text_input)['input_ids']) > max_len_token_vector:
            max_len_token_vector = len(tokenizer(text_input)['input_ids'])

    return max_len_raw, max_len_token_vector

max_len_raw_train, max_len_token_vector_train = get_max_tokenized_length(raw_train_ds['title'])
max_len_raw_test, max_len_token_vector_test = get_max_tokenized_length(raw_test_ds['title'])

print(f'max_len_raw_train = {max_len_raw_train}')
print(f'max_len_token_vector_train = {max_len_token_vector_train}')
print('')
print(f'max_len_raw_test = {max_len_raw_test}')
print(f'max_len_token_vector_test = {max_len_token_vector_test}')
print('')
max_len_token_vector = max(max_len_token_vector_train, max_len_token_vector_test)
print(f'max_len_token_vector (BOTH) = {max_len_token_vector}')

max_len_raw_train = 18
max_len_token_vector_train = 33

max_len_raw_test = 12
max_len_token_vector_test = 24

max_len_token_vector (BOTH) = 33


In [18]:
tokenizer

RobertaTokenizerFast(name_or_path='roberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [19]:
def preprocess_function(examples):
    global max_len_token_vector
    label = examples["sentiment"] 
    examples = tokenizer(examples["title"], truncation=True, padding="max_length", max_length=max_len_token_vector)
    
    # Change this to real number
    examples["label"] = float(label)
    return examples

## Perform 10-Fold Cross Validation

In [22]:
def tenfold_cross_val(train_ds, checkpoint):

    all_results = []

    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    for i, (train_index, test_index) in enumerate(kfold.split(train_ds['title'], train_ds['sentiment'])):
        
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")

        tokenized_cv_train = Dataset.from_dict(train_ds[train_index]).map(preprocess_function, remove_columns=['id', 'company', 'title', 'sentiment'])
        tokenized_cv_test = Dataset.from_dict(train_ds[test_index]).map(preprocess_function, remove_columns=['id', 'company', 'title', 'sentiment'])

        data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

        cv_train_dataset_tf = tokenized_cv_train.to_tf_dataset(
            columns=["attention_mask", "input_ids"],# "token_type_ids"],
            label_cols=["labels"],
            shuffle=False,
            collate_fn=data_collator,
            batch_size=4,
        )

        cv_test_dataset_tf = tokenized_cv_test.to_tf_dataset(
            columns=["attention_mask", "input_ids"],# "token_type_ids"],
            label_cols=["labels"],
            shuffle=False,
            collate_fn=data_collator,
            batch_size=4,
        )
    
        model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1) # num_labels=1 --> regression head after BERT layer (linear layer for output)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss="mse", metrics=['cosine_proximity'])
        model.fit(cv_train_dataset_tf, epochs=10)

        predicted_sentiments = model.predict(cv_test_dataset_tf)
        result = cosine_score(predicted_sentiments['logits'][:,0],  np.concatenate([y for x, y in cv_test_dataset_tf], axis=0))
        print(f'Fold {i} cosine_similarity = {result}\n\n')
        all_results.append(result)

    return all_results

In [23]:
cv_raw_data_cosine_similarity_all_folds = tenfold_cross_val(raw_train_ds, checkpoint)
cv_raw_data_cosine_similarity_avg = (sum(cv_raw_data_cosine_similarity_all_folds) / len(cv_raw_data_cosine_similarity_all_folds)) * 100

Fold 0:
  Train: index=[   0    1    2 ... 1139 1140 1141]
  Test:  index=[  10   23   31   44   51   56   70   86   88   96  101  109  113  128
  140  158  178  198  199  208  240  243  244  247  277  291  292  296
  298  306  318  319  323  331  333  336  342  344  355  361  362  367
  371  377  388  390  398  422  429  439  442  447  448  451  453  461
  482  493  494  497  514  528  529  530  535  538  558  599  605  622
  644  666  674  701  703  708  711  718  727  733  741  754  757  771
  774  777  793  810  814  827  844  875  893  909  922  924  933  940
  961  970  976 1001 1008 1011 1014 1046 1062 1077 1080 1094 1096 1107
 1119 1128 1133]


  0%|          | 0/1027 [00:00<?, ?ex/s]

  0%|          | 0/115 [00:00<?, ?ex/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
257/257 [==============================] - 28s 51ms/step - loss: 0.1176 - cosine_proximity: 0.4226
Epoch 2/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0593 - cosine_proximity: 0.7342
Epoch 3/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0468 - cosine_proximity: 0.7751
Epoch 4/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0433 - cosine_proximity: 0.7556
Epoch 5/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0306 - cosine_proximity: 0.8510
Epoch 6/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0255 - cosine_proximity: 0.8315
Epoch 7/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0224 - cosine_proximity: 0.8277
Epoch 8/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0213 - cosine_proximity: 0.8296
Epoch 9/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0222 - cosine_proximity: 0.8627
Epoch 10/1

...

Epoch 1/10
257/257 [==============================] - 28s 51ms/step - loss: 0.1029 - cosine_proximity: 0.4796
Epoch 2/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0578 - cosine_proximity: 0.6975
Epoch 3/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0457 - cosine_proximity: 0.7558
Epoch 4/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0370 - cosine_proximity: 0.7850
Epoch 5/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0309 - cosine_proximity: 0.7986
Epoch 6/10
257/257 [==============================] - 14s 53ms/step - loss: 0.0326 - cosine_proximity: 0.7870
Epoch 7/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0287 - cosine_proximity: 0.8084
Epoch 8/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0261 - cosine_proximity: 0.8006
Epoch 9/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0239 - cosine_proximity: 0.8317
Epoch 10/1

  0%|          | 0/1028 [00:00<?, ?ex/s]

  0%|          | 0/114 [00:00<?, ?ex/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
257/257 [==============================] - 27s 50ms/step - loss: 0.1320 - cosine_proximity: 0.3346
Epoch 2/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0648 - cosine_proximity: 0.7237
Epoch 3/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0542 - cosine_proximity: 0.7237
Epoch 4/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0379 - cosine_proximity: 0.7938
Epoch 5/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0330 - cosine_proximity: 0.8288
Epoch 6/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0282 - cosine_proximity: 0.8405
Epoch 7/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0269 - cosine_proximity: 0.8171
Epoch 8/10
257/257 [==============================] - 13s 51ms/step - loss: 0.0251 - cosine_proximity: 0.8424
Epoch 9/10
257/257 [==============================] - 13s 52ms/step - loss: 0.0241 - cosine_proximity: 0.8152
Epoch 10/1

In [24]:
cv_raw_data_cosine_similarity_avg

77.60886251926422

# DeBERTa
deberta-v3  
Cross Validation cosine_similarity = 80.45





## Imports

In [ ]:
!pip uninstall transformers
!pip uninstall sentencepiece

In [ ]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.26.0-py3-none-any.whl (6.3 MB)


## Preprocessing for input into DeBERTa

In [ ]:
checkpoint = "microsoft/deberta-v3-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def get_max_tokenized_length(ds):
    max_len_raw = 0
    max_len_token_vector = 0
    
    for text_input in ds:
        
        if len(text_input.split(" ")) > max_len_raw:
            max_len_raw = len(text_input.split(" "))
        
        if len(tokenizer(text_input)['input_ids']) > max_len_token_vector:
            max_len_token_vector = len(tokenizer(text_input)['input_ids'])

    return max_len_raw, max_len_token_vector

max_len_raw_train, max_len_token_vector_train = get_max_tokenized_length(raw_train_ds['title'])
max_len_raw_test, max_len_token_vector_test = get_max_tokenized_length(raw_test_ds['title'])

print(f'max_len_raw_train = {max_len_raw_train}')
print(f'max_len_token_vector_train = {max_len_token_vector_train}')
print('')
print(f'max_len_raw_test = {max_len_raw_test}')
print(f'max_len_token_vector_test = {max_len_token_vector_test}')
print('')
max_len_token_vector = max(max_len_token_vector_train, max_len_token_vector_test)
print(f'max_len_token_vector (BOTH) = {max_len_token_vector}')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


max_len_raw_train = 18
max_len_token_vector_train = 28

max_len_raw_test = 12
max_len_token_vector_test = 20

max_len_token_vector (BOTH) = 28


In [ ]:
tokenizer

DebertaV2TokenizerFast(name_or_path='microsoft/deberta-v3-base', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
def preprocess_function(examples):
    global max_len_token_vector
    label = examples["sentiment"] 
    examples = tokenizer(examples["title"], truncation=True, padding="max_length", max_length=max_len_token_vector)
    
    # Change this to real number
    examples["label"] = float(label)
    return examples

## Perform 10-Fold Cross Validation

In [ ]:
def tenfold_cross_val(train_ds, checkpoint):

    all_results = []

    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    for i, (train_index, test_index) in enumerate(kfold.split(train_ds['title'], train_ds['sentiment'])):
        
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")

        tokenized_cv_train = Dataset.from_dict(train_ds[train_index]).map(preprocess_function, remove_columns=['id', 'company', 'title', 'sentiment'])
        tokenized_cv_test = Dataset.from_dict(train_ds[test_index]).map(preprocess_function, remove_columns=['id', 'company', 'title', 'sentiment'])

        data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

        cv_train_dataset_tf = tokenized_cv_train.to_tf_dataset(
            columns=["attention_mask", "input_ids"],# "token_type_ids"],
            label_cols=["labels"],
            shuffle=False,
            collate_fn=data_collator,
            batch_size=4,
        )

        cv_test_dataset_tf = tokenized_cv_test.to_tf_dataset(
            columns=["attention_mask", "input_ids"],# "token_type_ids"],
            label_cols=["labels"],
            shuffle=False,
            collate_fn=data_collator,
            batch_size=4,
        )
    
        model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1, ignore_mismatched_sizes=True, from_pt=True) # num_labels=1 --> regression head after BERT layer (linear layer for output)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss="mse", metrics=['cosine_proximity'])
        model.fit(cv_train_dataset_tf, epochs=10)

        predicted_sentiments = model.predict(cv_test_dataset_tf)
        result = cosine_score(predicted_sentiments['logits'][:,0],  np.concatenate([y for x, y in cv_test_dataset_tf], axis=0))
        print(f'Fold {i} cosine_similarity = {result}\n\n')
        all_results.append(result)

    return all_results

In [ ]:
cv_raw_data_cosine_similarity_all_folds = tenfold_cross_val(raw_train_ds, checkpoint)
cv_raw_data_cosine_similarity_avg = (sum(cv_raw_data_cosine_similarity_all_folds) / len(cv_raw_data_cosine_similarity_all_folds)) * 100

Fold 0:
  Train: index=[   0    1    2 ... 1139 1140 1141]
  Test:  index=[  10   23   31   44   51   56   70   86   88   96  101  109  113  128
  140  158  178  198  199  208  240  243  244  247  277  291  292  296
  298  306  318  319  323  331  333  336  342  344  355  361  362  367
  371  377  388  390  398  422  429  439  442  447  448  451  453  461
  482  493  494  497  514  528  529  530  535  538  558  599  605  622
  644  666  674  701  703  708  711  718  727  733  741  754  757  771
  774  777  793  810  814  827  844  875  893  909  922  924  933  940
  961  970  976 1001 1008 1011 1014 1046 1062 1077 1080 1094 1096 1107
 1119 1128 1133]


  0%|          | 0/1027 [00:00<?, ?ex/s]

  0%|          | 0/115 [00:00<?, ?ex/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDebertaV2ForSequenceClassification: ['lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequen

Epoch 1/10
257/257 [==============================] - 59s 110ms/step - loss: 0.1083 - cosine_proximity: 0.4557
Epoch 2/10
257/257 [==============================] - 28s 109ms/step - loss: 0.0489 - cosine_proximity: 0.7400
Epoch 3/10
257/257 [==============================] - 28s 110ms/step - loss: 0.0335 - cosine_proximity: 0.8023
Epoch 4/10
257/257 [==============================] - 28s 109ms/step - loss: 0.0295 - cosine_proximity: 0.8218
Epoch 5/10
257/257 [==============================] - 28s 110ms/step - loss: 0.0225 - cosine_proximity: 0.8354
Epoch 6/10
257/257 [==============================] - 28s 111ms/step - loss: 0.0241 - cosine_proximity: 0.8315
Epoch 7/10
257/257 [==============================] - 28s 110ms/step - loss: 0.0213 - cosine_proximity: 0.8393
Epoch 8/10
257/257 [==============================] - 28s 110ms/step - loss: 0.0205 - cosine_proximity: 0.8374
Epoch 9/10
257/257 [==============================] - 28s 110ms/step - loss: 0.0202 - cosine_proximity: 0.8393
E

...

Epoch 1/10
257/257 [==============================] - 60s 111ms/step - loss: 0.1069 - cosine_proximity: 0.4621
Epoch 2/10
257/257 [==============================] - 29s 112ms/step - loss: 0.0505 - cosine_proximity: 0.7422
Epoch 3/10
257/257 [==============================] - 28s 111ms/step - loss: 0.0403 - cosine_proximity: 0.7870
Epoch 4/10
257/257 [==============================] - 28s 111ms/step - loss: 0.0297 - cosine_proximity: 0.7947
Epoch 5/10
257/257 [==============================] - 28s 111ms/step - loss: 0.0252 - cosine_proximity: 0.8142
Epoch 6/10
257/257 [==============================] - 28s 111ms/step - loss: 0.0242 - cosine_proximity: 0.8356
Epoch 7/10
257/257 [==============================] - 28s 111ms/step - loss: 0.0194 - cosine_proximity: 0.8259
Epoch 8/10
257/257 [==============================] - 28s 111ms/step - loss: 0.0201 - cosine_proximity: 0.8375
Epoch 9/10
257/257 [==============================] - 28s 111ms/step - loss: 0.0185 - cosine_proximity: 0.8337
E

  0%|          | 0/1028 [00:00<?, ?ex/s]

  0%|          | 0/114 [00:00<?, ?ex/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDebertaV2ForSequenceClassification: ['lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequen

Epoch 1/10
257/257 [==============================] - 60s 111ms/step - loss: 0.1054 - cosine_proximity: 0.4942
Epoch 2/10
257/257 [==============================] - 29s 111ms/step - loss: 0.0572 - cosine_proximity: 0.7121
Epoch 3/10
257/257 [==============================] - 28s 110ms/step - loss: 0.0424 - cosine_proximity: 0.7646
Epoch 4/10
257/257 [==============================] - 28s 111ms/step - loss: 0.0313 - cosine_proximity: 0.8132
Epoch 5/10
257/257 [==============================] - 29s 111ms/step - loss: 0.0289 - cosine_proximity: 0.8171
Epoch 6/10
257/257 [==============================] - 28s 110ms/step - loss: 0.0254 - cosine_proximity: 0.8366
Epoch 7/10
257/257 [==============================] - 28s 110ms/step - loss: 0.0218 - cosine_proximity: 0.8346
Epoch 8/10
257/257 [==============================] - 28s 110ms/step - loss: 0.0265 - cosine_proximity: 0.8366
Epoch 9/10
257/257 [==============================] - 28s 111ms/step - loss: 0.0216 - cosine_proximity: 0.8171
E

In [ ]:
cv_raw_data_cosine_similarity_avg

80.44909358024597